In [49]:
import pandas as ps
datasource1_dir = r'C:\Users\Maciej\Desktop\studia\przedmioty\BigData\projekt\zestaw2\input\datasource1\part-00000'
datasource4_dir = r'C:\Users\Maciej\Desktop\studia\przedmioty\BigData\projekt\zestaw2\input\datasource4\taxi_zone_lookup.csv'

In [67]:

trips_ps = ps.read_csv(datasource1_dir, header=None)
trips_ps.columns = ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'Passenger_count', 'Trip_distance', 'RateCodeID', 'Store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'Payment_type', 'Fare_amount', 'Extra', 'MTA_tax', 'Tip_amount', 'Tolls_amount', 'Improvement_surcharge', 'Total_amount']
trips_ps['month'] = trips_ps['tpep_pickup_datetime'].apply(lambda x: x[:7])
trips_ps['day'] = trips_ps['tpep_pickup_datetime'].apply(lambda x: x[9:11])
trips_ps = trips_ps.drop(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'RateCodeID', 'Store_and_fwd_flag', 'DOLocationID', 'Fare_amount', 'Extra', 'MTA_tax', 'Tip_amount', 'Tolls_amount', 'Improvement_surcharge'], axis=1)
taxi_zones_ps = ps.read_csv(datasource4_dir)
taxi_zones_ps = taxi_zones_ps.drop(['Zone', 'service_zone'], axis=1) 

In [69]:
trips_ps.head()

,Passenger_count,Trip_distance,PULocationID,Payment_type,Total_amount,month,day
0,1,5.99,231,1,33.36,2018-12,2
1,1,2.95,161,1,25.74,2018-12,6
2,2,1.23,237,1,9.96,2018-12,0
3,6,0.40,263,1,5.76,2018-12,9
4,1,2.00,236,1,11.15,2018-11,0


In [70]:
taxi_zones_ps.head()

,LocationID,Borough
0,1,EWR
1,2,Queens
2,3,Bronx
3,4,Manhattan
4,5,Staten Island


In [71]:
trips_filtered_ps = trips_ps

In [72]:
merged_ps = trips_filtered_ps.merge(taxi_zones_ps, left_on="PULocationID", right_on="LocationID")

In [74]:
agg_boroughs = merged_ps.groupby(["month", "Borough"]).agg(
    passengers=("Passenger_count", "sum"),
    total_amount=("Total_amount", "sum"),
    trip_distance=("Trip_distance", "sum"),
    count_trips=("month", "count")
).sort_values(by=["month", "Borough"])

In [85]:
filtered_agg_boroughs = agg_boroughs[agg_boroughs["count_trips"] >= 1000]

In [154]:
filtered_agg_boroughs.reset_index().head()

,month,Borough,passengers,total_amount,trip_distance,count_trips
0,2018-11,Manhattan,116482,1081771.32,171090.78,73546
1,2018-11,Queens,8013,230502.54,57147.29,5136
2,2018-11,Unknown,2124,26979.36,3729.72,1523
3,2018-12,Brooklyn,1606,19812.36,4255.26,1056
4,2018-12,Manhattan,118686,1072961.16,172797.77,74020


In [155]:
top_boroughs = filtered_agg_boroughs.sort_values(by=['month', 'Borough']).groupby('month').head(3).reset_index()

In [94]:
agg_days = merged_ps.groupby(["month", "Borough", "day"]).agg(
    count_trips=("month", "count")
).sort_values(by=["month", "Borough", "count_trips"], ascending=[True, True, False])

In [150]:
top_days = agg_days.groupby(["month", "Borough"]).head(3).reset_index()

In [148]:
top_days_collected = top_days.groupby(["month", "Borough"])[['day', 'count_trips']].apply(lambda x: x.values.tolist())

In [160]:
top_days_collected.reset_index().head()

,month,Borough,0
0,2008-12,Queens,"[[1 , 1]]"
1,2009-01,Brooklyn,"[[1 , 1]]"
2,2018-10,Manhattan,"[[1 , 2]]"
3,2018-11,Bronx,"[[2 , 20], [0 , 18], [8 , 15]]"
4,2018-11,Brooklyn,"[[0 , 128], [9 , 111], [1 , 109]]"


In [158]:
top_boroughs.merge(top_days_collected, on=['month', 'Borough'])

ValueError: Cannot merge a Series without a name